In [46]:
import gensim
import json
import logging

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

STEMMER = Mystem()

STOPWORDS_FILE = 'data/russian_stopwords.txt'
with open(STOPWORDS_FILE) as sf:
    STOPWORDS = [line.strip() for line in sf]

DATA_FILE = 'data/text_posts.json'
with open(DATA_FILE) as df:
    TEXTS = json.loads(df.read())
    
N_TOPICS = 5

# Препроцессинг данных


Определяем функции для токенизации

In [32]:
def preprocess(text, tokenize = RegexpTokenizer(r'\w+').tokenize, stopwords=STOPWORDS, stem=STEMMER.lemmatize):
    # TODO: add lemmatizer
    lowercase_words = (token.lower() for token in tokenize(text))
    no_stopwords = (word for word in lowercase_words if word not in stopwords)
    stemmed = (stem(word)[0] for word in no_stopwords)
    return list(stemmed)

In [33]:
tokens4texts = [preprocess(text) for text in TEXTS] # текст -> массив слов

In [54]:
id2word = gensim.corpora.Dictionary(tokens4texts)
id2word.filter_extremes(no_below=5, no_above=0.5)
print(id2word)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(9090 unique tokens: ['игуана', 'коллекционировать', 'буквально', 'ухажер', 'зашкаливать']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(11562 unique tokens: ['английски', 'настя', 'узор', 'буквально', 'выдавливать']...)
INFO:gensim.corpora.dictionary:built Dictionary(11629 unique tokens: ['английски', 'настя', 'узор', 'буквально', 'выдавливать']...) from 20407 documents (total 167020 corpus positions)
INFO:gensim.corpora.dictionary:discarding 7958 tokens: [('античный', 1), ('воплощать', 1), ('вымолвить', 1), ('кубик', 4), ('миронов', 2), ('приобнимать', 2), ('развеивать', 2), ('разжигать', 1), ('скульптура', 1), ('шарм', 2)]...
INFO:gensim.corpora.dictionary:keeping 3671 tokens which were in no less than 5 and no more than 10203 (=50.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary

Dictionary(3671 unique tokens: ['мужик', 'краска', 'попадать', 'буквально', 'зашкаливать']...)


Преобразуем тектсы в вектора:

In [55]:
corpus = [id2word.doc2bow(doc) for doc in tokens4texts]

In [ ]:
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=id2word, chunksize=50, update_every=1, passes=2)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online LDA training, 5 topics, 2 passes over the supplied corpus of 20407 documents, updating model once every 50 documents, evaluating perplexity every 500 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #50/20407
INFO:gensim.models.ldamodel:merging changes from 50 documents into a model of 20407 documents
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.015*vk + 0.014*работа + 0.014*http + 0.014*зарабатывать + 0.014*cc + 0.012*простой + 0.011*шаг + 0.010*падать + 0.009*месяц + 0.008*доллар
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.013*хотеть + 0.013*средство + 0.012*находить + 0.012*думать + 0.011*понимать + 0.011*мужчина + 0.011*женщина + 0.010*дело + 0.010*девушка + 0.008*любимый
INFO:gensim.models.ldamodel:topic #2 (0.200): 0.022*портить + 0.015

In [48]:
def print_top_terms(lda, num_terms=10):
    for i in range(0, lda.num_topics):
        terms = [term for val, term in lda.show_topic(i, num_terms)]
        print("topic #", str(i), ": ", " ".join(terms))  

In [57]:
print_top_terms(lda_model)

topic # 0 :  самый хороший счастие человек жизнь твой друг мой любовь давать
topic # 1 :  нравиться любовь парень девушка 3 2 выбирать дружба 1 хотеть
topic # 2 :  хотеть мужчина женщина девушка любить счастливый становиться парень должный ценить
topic # 3 :  друг человек жить думать скучать не потерять улыбаться прощать давать
topic # 4 :  любить говорить знать человек душа глаз сказать хотеться слово подруга
